In [2]:
from helpers import *

import sys

import bz2
import json

import pickle

import numpy as np
# import scipy

import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.pipeline import Pipeline

from sklearn.naive_bayes import MultinomialNB
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.linear_model import SGDClassifier
# from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import cross_val_score, cross_validate

%load_ext autoreload
%autoreload 2

## Finding the optimal amount of preprocessing

## Full data model training

In [3]:
# path = fixpath(QUOTES_LABELED_CLEANED)
path = fixpath(QUOTES_2020_LABELED_CLEANED)
df_raw = pd.read_json(path, orient='records', lines=True)
df_raw

,quoteID,quotation,speaker,date,numOccurrences,id,party_label,US_congress_bio_ID,quotation_clean
0,2020-01-16-000088,[ Department of Homeland Security ] was livid ...,sue myrick,2020-01-16 12:00:13,1,Q367796,R,M001134,department homeland security livid strongly ur...
1,2020-01-26-000499,a few of the candidates who will do better in ...,dave loebsack,2020-01-26 13:21:36,11,Q771586,D,L000565,candidate better part world
2,2020-01-26-040663,"The generational thing is important, quite hon...",dave loebsack,2020-01-26 13:21:36,11,Q771586,D,L000565,generational thing important quite honestly th...
3,2020-01-20-000982,a host of other protections,debbie lesko,2020-01-20 15:32:48,1,Q16731415,R,L000589,host protection
4,2020-01-09-060095,"No, are you kidding? The way that this place l...",debbie lesko,2020-01-09 23:15:21,1,Q16731415,R,L000589,kidding way place leak around
...,...,...,...,...,...,...,...,...,...
349141,2020-02-10-100845,We're just worried about making sure we keep t...,sherrie sprenger,2020-02-10 00:00:00,1,Q7495360,R,None,worried making sure keep balance expression ri...
349142,2020-03-16-079753,We've all embraced strict proper hygiene proce...,robert abrams,2020-03-16 12:00:00,2,Q2156314,D,None,embraced strict proper hygiene procedure heard...
349143,2020-01-13-091997,What's important is that we keep moving forward.,laurie jinkins,2020-01-13 19:51:15,1,Q6501617,D,None,important keep moving forward
349144,2020-02-20-093793,"When they want a bill, they bring the bill to ...",j.t. wilcox,2020-02-20 22:12:45,1,Q6104393,R,None,want bill bring bill floor


In [4]:
df = df_raw.copy()
df.drop(['numOccurrences','US_congress_bio_ID'], axis=1, inplace=True)
df

,quoteID,quotation,speaker,date,id,party_label,quotation_clean
0,2020-01-16-000088,[ Department of Homeland Security ] was livid ...,sue myrick,2020-01-16 12:00:13,Q367796,R,department homeland security livid strongly ur...
1,2020-01-26-000499,a few of the candidates who will do better in ...,dave loebsack,2020-01-26 13:21:36,Q771586,D,candidate better part world
2,2020-01-26-040663,"The generational thing is important, quite hon...",dave loebsack,2020-01-26 13:21:36,Q771586,D,generational thing important quite honestly th...
3,2020-01-20-000982,a host of other protections,debbie lesko,2020-01-20 15:32:48,Q16731415,R,host protection
4,2020-01-09-060095,"No, are you kidding? The way that this place l...",debbie lesko,2020-01-09 23:15:21,Q16731415,R,kidding way place leak around
...,...,...,...,...,...,...,...
349141,2020-02-10-100845,We're just worried about making sure we keep t...,sherrie sprenger,2020-02-10 00:00:00,Q7495360,R,worried making sure keep balance expression ri...
349142,2020-03-16-079753,We've all embraced strict proper hygiene proce...,robert abrams,2020-03-16 12:00:00,Q2156314,D,embraced strict proper hygiene procedure heard...
349143,2020-01-13-091997,What's important is that we keep moving forward.,laurie jinkins,2020-01-13 19:51:15,Q6501617,D,important keep moving forward
349144,2020-02-20-093793,"When they want a bill, they bring the bill to ...",j.t. wilcox,2020-02-20 22:12:45,Q6104393,R,want bill bring bill floor


In [5]:
df['party_label'].value_counts()

D    218894
R    130252
Name: party_label, dtype: int64

### Filtering politcally insignificant quotes 

In [6]:
def filter_relevant_quotes_basic(df):
    lengths_of_df = df['quotation_clean'].apply(lambda x: len(x))

    # df = df[np.logical_and(lengths_of_df > lengths_of_df.quantile(0.3), lengths_of_df < lengths_of_df.quantile(0.9))]
    df = df[lengths_of_df > lengths_of_df.quantile(0.1)]

    return df

In [7]:
df_filt = filter_relevant_quotes_basic(df)
df_filt

,quoteID,quotation,speaker,date,id,party_label,quotation_clean
0,2020-01-16-000088,[ Department of Homeland Security ] was livid ...,sue myrick,2020-01-16 12:00:13,Q367796,R,department homeland security livid strongly ur...
1,2020-01-26-000499,a few of the candidates who will do better in ...,dave loebsack,2020-01-26 13:21:36,Q771586,D,candidate better part world
2,2020-01-26-040663,"The generational thing is important, quite hon...",dave loebsack,2020-01-26 13:21:36,Q771586,D,generational thing important quite honestly th...
4,2020-01-09-060095,"No, are you kidding? The way that this place l...",debbie lesko,2020-01-09 23:15:21,Q16731415,R,kidding way place leak around
5,2020-01-03-001237,A lot of Americans [ were ] upset that this wa...,julián castro,2020-01-03 00:00:00,Q970720,D,lot american upset happening lot people know g...
...,...,...,...,...,...,...,...
349141,2020-02-10-100845,We're just worried about making sure we keep t...,sherrie sprenger,2020-02-10 00:00:00,Q7495360,R,worried making sure keep balance expression ri...
349142,2020-03-16-079753,We've all embraced strict proper hygiene proce...,robert abrams,2020-03-16 12:00:00,Q2156314,D,embraced strict proper hygiene procedure heard...
349143,2020-01-13-091997,What's important is that we keep moving forward.,laurie jinkins,2020-01-13 19:51:15,Q6501617,D,important keep moving forward
349144,2020-02-20-093793,"When they want a bill, they bring the bill to ...",j.t. wilcox,2020-02-20 22:12:45,Q6104393,R,want bill bring bill floor


In [8]:
df_filt['party_label'].value_counts()

D    194756
R    117083
Name: party_label, dtype: int64

In [9]:
df_filt['quotation_clean'].apply(lambda x: len(x)).min()

24

### Balancing data

In [10]:

df_filt = downsample(df_filt, 'party_label')

In [11]:
df_filt['party_label'].value_counts()

D    117083
R    117083
Name: party_label, dtype: int64

In [12]:
df_filt

,quoteID,quotation,speaker,date,id,party_label,quotation_clean
0,2020-04-16-028496,It would be a huge mistake to restart too earl...,bill de blasio,2020-04-16 14:05:20,Q4911497,D,would huge mistake restart early would huge mi...
1,2020-01-03-013634,"evil man whose death we do not mourn,",chellie pingree,2020-01-03 16:51:33,Q457243,D,evil man whose death mourn
2,2020-03-29-000363,a little crispy around the edges,melissa hortman,2020-03-29 00:00:00,Q6812738,D,little crispy around edge
3,2020-01-09-077904,"The cost of healthcare is just too damn high, ...",gavin newsom,2020-01-09 15:15:05,Q461391,D,cost healthcare damn high california fighting ...
4,2020-02-27-049540,"My mother would say, `The world is yours, grab...",herb wesson,2020-02-27 00:00:00,Q16151544,D,mother would say world grab tail shake hell ba...
...,...,...,...,...,...,...,...
234161,2020-04-02-047841,pervasiveness and seriousness of the FISA appl...,jim jordan,2020-04-02 10:00:49,Q186215,R,pervasiveness seriousness fisa application def...
234162,2020-02-21-012069,"Don't think you've been forgotten, don't think...",steve bannon,2020-02-21 19:53:14,Q16146870,R,think forgotten think forgotten man forgotten ...
234163,2020-01-03-074389,We just want to make sure our tax money is spe...,pete flores,2020-01-03 06:00:00,Q56678374,R,want make sure tax money spent appropriately
234164,2020-03-26-014365,failed to protect students and address their c...,betsy devos,2020-03-26 22:21:20,Q4898286,R,failed protect student address complaint sexua...


## Model Training

In [13]:
df_mini = df_filt.sample(100000)
# df_mini = df_filt.sample(frac=1)

In [14]:
X = df_mini['quotation_clean'].values

y = df_mini['party_label'].values
y = np.array([0.0 if label=='R' else 1.0 for label in y])

In [15]:
split_frac = 0.9

bound = round(len(X)*split_frac)

X_train = X[:bound]
y_train = y[:bound]

X_test = X[bound:]
y_test = y[bound:]

In [16]:
X_train.shape

(90000,)

In [16]:
def vectorize_with_file(X, vectorizer):
    path_temp = TEMP_FILE

    with open(path_temp, 'w') as d_file:
        d_file.writelines(X + '\n')

    with open(path_temp, 'r') as s_file:
        X_vect=vectorizer.fit_transform(s_file)

    return X_vect

In [26]:
# Vectorize by writing to file
# vectorizer=TfidfVectorizer(ngram_range=(1,3))
# X_vect = vectorize_with_file(X_train, vectorizer)

In [18]:
# Vectorize in memory
vectorizer=TfidfVectorizer()
X_vect = vectorizer.fit_transform(X_train)

In [19]:
# with open(TEMP_FOLDER + 'quotes_vectorized_ngram=(1,3).pkl', 'wb') as file:
#     pickle.dump(X_vect, file)

In [20]:
X_test_vect = vectorizer.transform(X_test)

In [21]:
X_vect.shape

(90000, 29034)

In [33]:
# with open(TEMP_FOLDER + 'vectorizer_ngram=(1,3)-multinomialnb.pkl', 'wb') as file:
#     pickle.dump(vectorizer, file)

In [ ]:
# with open(TEMP_FOLDER + 'vectorizer_ngram=(1,3)-multinomialnb.pkl', 'rb') as file:
#     vectorizer_loaded = pickle.load(file)

In [22]:
clf = MultinomialNB()

clf.fit(X_vect, y_train)
clf.score(X_test_vect, y_test)


0.6555

In [32]:
# with open(TEMP_FOLDER + 'model-tfidf_ngram=(1,3)-multinomialnb.pkl', 'wb') as file:
#     pickle.dump(clf, file)

In [24]:
# with open(TEMP_FOLDER + 'model-tfidf_ngram=(1,3)-multinomialnb.pkl', 'rb') as file:
#     clf_loaded = pickle.load(file)

----------

In [25]:
clf_loaded.score(X_test_vect, y_test)

0.7012336262001888

In [ ]:
TfidfVectorizer(TEMP_FILE, )

In [28]:
pipeline_simple = Pipeline([
        ('vect', TfidfVectorizer(ngram_range=(1,1), max_features=100_000)),  
        ('clf', MultinomialNB())
    ],
    verbose=True
    )

In [29]:
# Run cross validation

scoring=['accuracy', 'precision', 'recall', 'f1']
res = cross_validate(pipeline_simple, X, y, scoring=scoring, cv=2, n_jobs=2)
print_cross_validate_results(res)

[Pipeline] .............. (step 1 of 2) Processing vect, total=   2.1s
[Pipeline] ............... (step 2 of 2) Processing clf, total=   0.0s
[Pipeline] .............. (step 1 of 2) Processing vect, total=   2.1s
[Pipeline] ............... (step 2 of 2) Processing clf, total=   0.0s
	fit_time             - 	avg: 2.138	std: 0.021
	score_time           - 	avg: 2.696	std: 0.015
	test_accuracy        - 	avg: 0.662	std: 0.001
	test_precision       - 	avg: 0.663	std: 0.001
	test_recall          - 	avg: 0.661	std: 0.001
	test_f1              - 	avg: 0.662	std: 0.000


In [30]:
split_frac = 0.9

bound = round(len(X)*split_frac)

X_train = X[:bound]
y_train = y[:bound]

X_test = X[bound:]
y_test = y[bound:]

In [31]:
X_train.shape, X_test.shape

((210749,), (23417,))

In [32]:
pipeline_simple.fit(X_train, y_train)

[Pipeline] .............. (step 1 of 2) Processing vect, total=   3.5s
[Pipeline] ............... (step 2 of 2) Processing clf, total=   0.1s


Pipeline(steps=[('vect', TfidfVectorizer(max_features=100000)),
                ('clf', MultinomialNB())],
         verbose=True)

In [33]:
pipeline_simple.score(X_test, y_test)

0.6719050262629713

In [35]:
pipeline_simple.predict(X[0:10])

array([1., 1., 1., 0., 0., 0., 0., 1., 1., 1.])